In [38]:
from bs4 import BeautifulSoup
import platform
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.font_manager as fm
import pandas as pd

In [39]:
class NaverNews:
    url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge'

    def __init__(self, search, sort, name=''):
        search = '&query=' + str(search)
        sort = '&sort=' + str(sort)
        self.search = search
        self.sort = sort
        self.name = name
        self.search_url = NaverNews.url + search + sort

    
    def find_tum_text(self, maxpage, tumClass = 'api_txt_lines dsc_txt_wrap'):

        result_list = []

        for num in range(1, maxpage+1):
            page_num = '&start=' + str(10*num-9)
            full_url = self.search_url + page_num 
            req = requests.get(full_url)
            bs = BeautifulSoup(req.text, "html.parser")

            tums = bs.find_all('a', {'class': tumClass})

            for tum in tums:
                result_list.append(tum.text)
        
        return result_list

In [40]:
def get_html(url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None

        return BeautifulSoup(req.text, "html.parser")

In [41]:
def daum_web_find(search, maxpage=1):  
    url = 'https://search.daum.net/search?nil_suggest=btn&w=fusion&DA=SBC&q=' + search
    result = []

    for i in range(1,maxpage+1):
        url2 = url + '&p=' + str(i)
        html = get_html(url2)

        a = html.find_all('p',{'class': 'desc'})
        for i in a:
            result.append(i.text)    
    
    return result

In [42]:
def to_txt(list, filename):
    with open(filename,'w',encoding='UTF-8') as f:
        for txt in list:
            f.write(txt+'\n')

In [43]:
def make_wordcloud(filepath):
    daum_pos = open(filepath, encoding='utf-8').read()
    okt = Okt()

    sentences_tag = []
    sentences_tag = okt.pos(daum_pos)

    noun_adj_list = []
    for word, tag in sentences_tag:
        if tag in ["Noun", "Adjective"]:
            noun_adj_list.append(word)

    counts = Counter(noun_adj_list)
    tags = counts.most_common(35)

    if platform.system() == "Windows":
        path = r"c:\Windows\Fonts\malgun.ttf"
        plt.rc('font', family='Malgun Gothic')
        
    elif platform.system() == "Darwin":
        path = r"/System/Library/Fonts/AppleGothic"
        plt.rc('font', familly='AppleGothic')
    else:
        path = r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf"


    wc = WordCloud(font_path=path, background_color="white", max_font_size=80)
    cloud = wc.generate_from_frequencies(dict(tags))


    plt.figure(figsize=(10, 8))
    plt.axis("off")
    plt.imshow(cloud, interpolation="bilinear")
    plt.show()

In [44]:
# db에 저장하기 위해 튜플데이터 만들어주는 함수
def make_tuple_data(data, maintag, subtag):

    if type(data) == list:
        data = " ".join(data)
    else:
        data = open(data, encoding="utf-8").read()

    okt = Okt()

    sentences_tag = []
    sentences_tag = okt.pos(data)

    noun_adj_list = []
    for word, tag in sentences_tag:
        if tag in ["Noun", "Adjective"]:
            noun_adj_list.append(word)

    counts = Counter(noun_adj_list)
    tags = counts.most_common(40)
    result = []
    for i in tags:
        if i[0] == maintag:
            pass
        else:
            i = list(i)
            i.insert(0,subtag)
            result.append(tuple(i))


    return result

****
## <span style='color:royalBlue'>데이터 크롤링

In [45]:
a = make_tuple_data('./네이버_노인+혐오.txt', '노인', '혐오')
a = pd.DataFrame(a)
a.to_csv('./네이버_노인+혐오.csv', header=False)

In [46]:
a = make_tuple_data('./다음_노인+혐오.txt', '노인', '혐오')
a = pd.DataFrame(a)
a.to_csv('./다음_노인+혐오.csv', header=False)

In [47]:
# result = daum_web_find('노인+종업원', 70)
# to_txt(result, '다음_노인+종업원.txt')

In [48]:
# result = NaverNews('노인+종업원', 0)
# result = result.find_tum_text(70)
# to_txt(result, '네이버_노인+종업원.txt')

In [49]:
# make_wordcloud('./다음_노인+일자리.txt'), make_wordcloud('./네이버_노인+일자리.txt')